# Training

In [1]:
import sys, os
import pandas as pd
import numpy as np
import setting
import pathSetting
import torch
sys.path.append("../../..")

from Clust.clust.transformation.type.DFToNPArray import transDFtoNP, trans_df_to_np, trans_df_to_np_inf
from Clust.clust.ML.tool import data as ml_data
from Clust.clust.ML.tool import model as ml_model
from Clust.clust.ML.tool import clean as ml_clean
from Clust.clust.ML.tool import meta as ml_meta

from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_client_v2 as influx_Client
from Clust.clust.ingestion.mongo.mongo_client import MongoClient

import torch

db_client = influx_Client.InfluxClient(ins.CLUSTDataServer2)
mongo_client = MongoClient(ins.CLUSTMetaInfo2)

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{device}" " is available.")

cpu is available.


In [2]:
# Set model methods i.e., 'LSTM_cf', 'GRU_cf', 'CNN_1D_cf', 'LSTM_FCNs_cf', 'FC_cf' 
model_method = 'LSTM_cf'

# get integrated data name
bucket_name = 'integration'

# scaler path
scalerPath = './scaler/'

In [3]:
get_ms_list = db_client.measurement_list(bucket_name)
get_ms_list

['classification_actionPattern_testX_cleanLevel0',
 'classification_actionPattern_testy_cleanLevel0',
 'classification_actionPattern_trainX_cleanLevel0',
 'classification_actionPattern_trainy_cleanLevel0',
 'forecasting_Hs2SwineFarmWithWeatherTime_test_cleanLevel0',
 'forecasting_Hs2SwineFarmWithWeatherTime_test_cleanLevel4',
 'forecasting_Hs2SwineFarmWithWeatherTime_train_cleanLevel0',
 'forecasting_Hs2SwineFarmWithWeatherTime_train_cleanLevel4',
 'forecasting_gunwiStrawberryWeather_test_cleanLevel0',
 'forecasting_gunwiStrawberryWeather_test_cleanLevel4',
 'forecasting_gunwiStrawberryWeather_train_cleanLevel0',
 'forecasting_gunwiStrawberryWeather_train_cleanLevel4',
 'forecasting_strawberryOpen_test_cleanLevel0',
 'forecasting_strawberryOpen_test_cleanLevel4',
 'forecasting_strawberryOpen_train_cleanLevel0',
 'forecasting_strawberryOpen_train_cleanLevel4',
 'regression_energy_testX_cleanLevel0',
 'regression_energy_testX_cleanLevel4',
 'regression_energy_testy_cleanLevel0',
 'regres

In [4]:
collection_list = mongo_client.get_collection_list(bucket_name)
collection_list

['forecasting_strawberryOpen',
 'regression_energy',
 'forecasting_Hs2SwineFarmWithWeatherTime',
 'forecasting_gunwiStrawberryWeather',
 'classification_actionPattern']

In [5]:
data_name_X = 'classification_actionPattern_trainX_cleanLevel0'
dataX = db_client.get_data(bucket_name, data_name_X)

# datay
data_name_y = 'classification_actionPattern_trainy_cleanLevel0'
datay = db_client.get_data(bucket_name, data_name_y)

dataset_name = collection_list[4]
data_meta = mongo_client.get_document_by_json('integration', dataset_name, {'data_name':data_name_X})[0]
clean_level = data_meta["clean_level"]
integration_freq_sec = data_meta["integration_param"]["integration_frequency"]

In [6]:
# 2 Training Data Preparation
# 2-1
featureListX= list(dataX.columns)
featureListy= list(datay.columns)

# 2-2
# cleanTrainDataParam = 'NoClean'#  Classification, Regression과 같이 X, y가 분리된 경우에는 현재 고정해서 사용해야함
# cleanTrainDataParam = clean_level --> clean_level로 변경

# 2-2-1 cleanTrainDataParam == Clean 일 경우
NaNProcessingParam ={
    "feature_cycle":'Day',
    "feature_cycle_times":1,
    "NanInfoForCleanData":{'type':'num', 'ConsecutiveNanLimit':3, 'totalNaNLimit':30000}
}
# 2-3
scalerParam='scale'
scaleMethod='minmax'

# 2-4
splitRatio = 0.8
mode = 'Classification'

# 2-5
scalerRootPath_X = os.path.join(scalerPath, data_name_X, str(clean_level))
scalerRootPath_y = os.path.join(scalerPath, data_name_y, str(clean_level))
train_X, val_X, X_scalerFilePath = ml_data.get_train_val_data(dataX, featureListX, scalerRootPath_X, splitRatio, scalerParam, scaleMethod, mode)
train_y, val_y, y_scalerFilePath = ml_data.get_train_val_data(datay, featureListy, scalerRootPath_y, splitRatio, "NoScale", scaleMethod, mode)

['col_0', 'col_1', 'col_2', 'col_3', 'col_4', 'col_5', 'col_6', 'col_7', 'col_8']
Make New scaler File


In [7]:
window_num = 0
dim = 3

if model_method == "FC_cf":
    dim = 2

if type(train_X) !=  np.ndarray:
    train_X, train_y = transDFtoNP(train_X, train_y, window_num, dim)
    val_X, val_y = transDFtoNP(val_X, val_y, window_num, dim)

input_size = train_X.shape[1]
if dim != 2:
    seq_len = train_X.shape[2] # seq_length

../../../../Clust/clust/transformation/type/DFToNPArray.py:38: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  dfX_partial = dfX[startDate:endDate]
../../../../Clust/clust/transformation/type/DFToNPArray.py:39: FutureWarning: Indexing a timezone-aware DatetimeIndex with a timezone-naive datetime is deprecated and will raise KeyError in a future version.  Use a timezone-aware object instead.
  dfy_partial = dfy[startDate:endDate]


In [8]:
print(type(train_X), train_X.shape)
print(train_X)

<class 'numpy.ndarray'> (5881, 9, 128)
[[[0.48670847 0.49064098 0.49030017 ... 0.48618388 0.48654927 0.48725925]
  [0.58418424 0.58238032 0.58339245 ... 0.57961972 0.58022402 0.58086851]
  [0.58408219 0.58390294 0.58113932 ... 0.5623219  0.56221218 0.56029954]
  ...
  [0.55528389 0.55904341 0.55874125 ... 0.55739637 0.55774432 0.55841957]
  [0.52112852 0.51982151 0.52084739 ... 0.52082385 0.52129699 0.52180391]
  [0.59656275 0.59750528 0.59627805 ... 0.59569013 0.59551497 0.59384219]]

 [[0.48706899 0.48843397 0.48777409 ... 0.48480219 0.48547545 0.48600567]
  [0.57752643 0.57632005 0.57591419 ... 0.57718512 0.57825884 0.57644752]
  [0.55018704 0.5509113  0.55055067 ... 0.56110973 0.55922348 0.55983588]
  ...
  [0.55754876 0.55887337 0.55827319 ... 0.55602371 0.55666594 0.55717266]
  [0.52085729 0.51984573 0.51949623 ... 0.51905306 0.51998268 0.51851992]
  [0.59484945 0.59533603 0.59491252 ... 0.59334502 0.59180443 0.59234438]]

 [[0.48803162 0.48753947 0.48647128 ... 0.48712912 0.4878

In [9]:
# RNN models (RNN, LSTM, GRU) parameters
if model_method == 'LSTM_cf' or model_method == 'GRU_cf':
    modelParameter = {
        'input_size': input_size,
        'seq_len': seq_len,
        'num_classes': 6,
        'num_layers': 2,  # recurrent layers의 수, int(default: 2, 범위: 1 이상)
        'hidden_size': 64,  # hidden state의 차원, int(default: 64, 범위: 1 이상)
        'dropout': 0.1,  # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
        'bidirectional': True  # 모델의 양방향성 여부, bool(default: True)   
    }
    if model_method == 'LSTM_cf':
        modelParameter['rnn_type'] = 'lstm'
    else:
        modelParameter['rnn_type'] = 'gru'
        
# CNN_1D model parameters
elif model_method == 'CNN_1D_cf':
    modelParameter = {
    'input_size': input_size,
    'seq_len': seq_len,
    'num_classes': 6,
    'output_channels': 64, # convolution layer의 output channel, int(default: 64, 범위: 1 이상, 2의 지수로 설정 권장)
    'kernel_size': 3, # convolutional layer의 filter 크기, int(default: 3, 범위: 3 이상, 홀수로 설정 권장)
    'stride': 1, # convolution layer의 stride 크기, int(default: 1, 범위: 1 이상)
    'padding': 0, # padding 크기, int(default: 0, 범위: 0 이상)
    'drop_out': 0.1 # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
    }
    
# LSTM_FCNs model parameters
elif model_method == 'LSTM_FCNs_cf':
    modelParameter = {
    'input_size': input_size,
    'seq_len': seq_len,
    'num_classes': 6,
    'num_layers': 1,  # recurrent layers의 수, int(default: 1, 범위: 1 이상)
    'lstm_drop_out': 0.4, # LSTM dropout 확률, float(default: 0.4, 범위: 0 이상 1 이하)
    'fc_drop_out': 0.1 # FC dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
    }
    
# FC model parameters
elif model_method == 'FC_cf':
    modelParameter = {
    'input_size': input_size,
    'num_classes': 6,
    'drop_out': 0.1, # dropout 확률, float(default: 0.1, 범위: 0 이상 1 이하)
    'bias': True# bias 사용 여부, bool(default: True)
    }

trainParameter = {
    'lr': 1e-4,
    'weight_decay': 1e-6, 
    'device': 'cpu', 
    'n_epochs': 5, 
    'batch_size': 16
}

In [10]:
modelTags =["action", "sensor", "classification", "pattern"]
trainDataType = "timeseries"
modelPurpose = "classification"

# # 2
trainDataInfo = data_meta

# 3. 모델을 저장할 파일 패스를 생성한다.
model_name = None
if model_name is None:
    model_name = dataset_name + '_' + model_method + '_cleanLevel' + str(clean_level)
else:
    pass

print(model_name)

from Clust.clust.transformation.general.dataScaler import encode_hash_style
trainParameter_encode =  encode_hash_style(str(trainParameter))
trainDataPathList = [model_name, data_name_X, trainParameter_encode]
modelFilePath = ml_model.get_model_file_path(trainDataPathList, model_method)

classification_actionPattern_LSTM_cf_cleanLevel0
./Models/LSTM_cf/classification_actionPattern_LSTM_cf_cleanLevel0/classification_actionPattern_trainX_cleanLevel0/035b06e2d9df4a58aa1dd8622746dd1f/model.pkl


In [11]:
from Clust.clust.ML.classification.train import ClassificationTrain as CML

cml = CML()
cml.set_param(trainParameter)
cml.set_model(model_method, modelParameter)
cml.set_data(train_X, train_y, val_X, val_y)
cml.train()
cml.save_best_model(modelFilePath)

Start training model
=================================11111111111111111111111111111111111111111111
cpu

Epoch 1/5


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
model_meta = {
    "trainDataInfo": trainDataInfo,
    "featureList": featureListX,
    "target": featureListy,
    "trainDataType": trainDataType,
    "modelPurpose": modelPurpose,
    "model_method": model_method,
    "modelTags": modelTags,
    "trainDataName": [data_name_X,data_name_y],
    "cleanTrainDataParam":cleanTrainDataParam,
    "NaNProcessingParam":NaNProcessingParam,
    "scalerParam": scalerParam,
    "trainParameter": trainParameter,
    "modelParameter": modelParameter,
    
    "files":{
            "modelFile":{
                    "fileName":"model.pth",
                    "filePath":modelFilePath
                },
            "XScalerFile":{
                    "fileName":"scaler.pkl",
                    "filePath":X_scalerFilePath       
                },
            "yScalerFile":{
                    "fileName":"scaler.pkl",
                    "filePath":y_scalerFilePath       
                }
        }
}

modelInfoMeta = ml_meta.save_model_meta_data(mongo_client, model_meta)